:::tip Previous: [Get started](/docs/expression_language/get_started)
:::

LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

We laid out the main value props in the [overview](/docs/expression_language) and went through two simple use cases in the [get started](/docs/expression_language/get_started) section. To really understand why LCEL and LangChain in general are useful, it's also helpful to think about how we might recreate similar functionality without them.

Let's try this with our [basic example](/docs/expression_language/get_started#basic_example) from the get started page:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

## Invoke
In the simplest case, we just want to be able to pass in a topic string and get back a joke string:


<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
import openai


def manual_chain(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    message = {"role": "user", "content": prompt_value}
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[message],
    )
    return response.choices[0].message.content

</TabItem>

<TabItem value="lcel" label="LCEL" default>

In [ ]:
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

</TabItem>
</Tabs>

## Stream
If we want to stream results instead, we'll need to change our function:

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
from typing import Iterator


def manual_chain_stream(topic: str) -> Iterator[str]:
    prompt_value = f"Tell me a short joke about {topic}"
    message = {"role": "user", "content": prompt_value}
    client = openai.OpenAI()
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[message],
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

</TabItem>

<TabItem value="lcel" label="LCEL" default>

In [ ]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

</TabItem>
</Tabs>


## Batch

If we want to run on a batch of inputs in parallel, we'll again need a new function:

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
from concurrent.futures import ThreadPoolExecutor


def manual_chain_batch(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(manual_chain, topics))

</TabItem>

<TabItem value="lcel" label="LCEL" default>

In [ ]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

</TabItem>
</Tabs>

## Async

If you needed an asynchronous version:

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [47]:
async def manual_chain_async(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    message = {"role": "user", "content": prompt_value}
    client = openai.AsyncOpenAI()
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[message]
    )
    return response.choices[0].message.content

</TabItem>

<TabItem value="lcel" label="LCEL">

```python
chain.ainvoke("ice cream)
```

</TabItem>
</Tabs>

## LLM instead of chat model

If we want to use a completion endpoint instead of a chat endpoint: 

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
def manual_chain_completion(topic: str) -> str:
    prompt_value = f"Tell me a short joke about {topic}"
    client = openai.OpenAI()
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt_value,
    )
    return response.choices[0].text

</TabItem>

<TabItem value="lcel" label="LCEL" default>

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)


If we wanted, we could even make the choice of chat model or llm runtime configurable

In [ ]:
from langchain_core.runnables import ConfigurableField


configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="chat_openai", 
    openai=llm
)
configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

In [ ]:
# Uses chat model by default
configurable_chain.invoke("ice cream")
# Uses LLM
configurable_chain.invoke(
    "ice cream", 
    config={"configurable": {"model": "openai"}}
)

</TabItem>
</Tabs>

## Different model provider

If we want to use Anthropic instead of OpenAI: 

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
import anthropic


def manual_chain_anthropic(topic: str) -> str:
    prompt_value = (
        f"Human:\n\nTell me a short joke about {topic}\n\nAssistant:"
    )
    client = anthropic.Anthropic()
    response = client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    return response.completion

</TabItem>

<TabItem value="lcel" label="LCEL" default>

In [ ]:
from langchain.chat_models import ChatAnthropic


anthropic_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | ChatAnthropic(model="claude-2")
    | output_parser
)
anthropic_chain.invoke("ice cream")

</TabItem>
</Tabs>

## Logging

If we want to log our intermediate results (we'll `print` here for illustrative purposes):

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
def manual_chain_anthropic_logging(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = (
        f"Human:\n\nTell me a short joke about {topic}\n\nAssistant:"
    )
    print(f"Formatted prompt: {prompt_value}")
    client = anthropic.Anthropic()
    response = client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    print(f"Output: {response.completion}")
    return response.completion

</TabItem>

<TabItem value="lcel" label="LCEL" default>

By turning on LangSmith, every step of every chain is automatically logged. We set these environment variables:

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

</TabItem>
</Tabs>

## Fallbacks

If you wanted to add retry or fallback logic:

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
def manual_chain_with_fallback(topic: str) -> str:
    try:
        return manual_chain(topic)
    except Exception:
        return manual_chain_anthropic(topic)

</TabItem>

<TabItem value="lcel" label="LCEL" >

In [ ]:
fallback_chain = chain.with_fallbacks([anthropic_chain])

</TabItem>
</Tabs>

## Full code comparison

<Tabs>
  <TabItem value="without_lcel" label="Without LCEL" default>

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import Iterator, List, Tuple

import openai

prompt_template = "Tell me a short joke about {topic}"


def manual_chain(topic: str, *, model: str = "chat_openai") -> str:
    print(f"Input: {topic}")
    prompt_value = prompt_template.format(topic=topic)

    if model == "chat_openai":
        print(f"Full prompt: {prompt_value}")
        message = {"role": "user", "content": prompt_value}
        response = openai.OpenAI().chat.completions.create(
            model="gpt-3.5-turbo", 
            messages=[message],
        )
        output = response.choices[0].message.content
    elif model == "openai":
        print(f"Full prompt: {prompt_value}")
        response = openai.OpenAI().completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt_value,
        )
        output = response.choices[0].text
    elif model == "anthropic":
        prompt_value = f"Human:\n\n{prompt_value}\n\nAssistant:"
        print(f"Full prompt: {prompt_value}")
        response = anthropic.Anthropic().completions.create(
            model="claude-2",
            prompt=prompt_value,
            max_tokens_to_sample=256,
        )
        output = response.completion
    else:
        raise ValueError(
            f"Invalid model {model}." 
            "Should be one of chat_openai, openai, anthropic."
        )
    print(f"Output: {output}")
    return output


def manual_chain_with_fallbacks(
    topic: str, 
    *, 
    model: str = "chat_openai", 
    fallbacks: Tuple[str] = ("anthropic",)
) -> str:
    for fallback in fallbacks:
        try:
            return manual_chain(topic, model=model)
        except Exception as e:
            print(f"Error {e}")
            model = fallback
    raise e


def manual_chain_batch(
    topics: List[str],
    *,
    model: str = "chat_openai",
    fallbacks: Tuple[str] = ("anthropic",),
) -> List[str]:
    models = [model] * len(topics)
    fallbacks_list = [fallbacks] * len(topics)
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(
            executor.map(
                manual_chain_with_fallbacks, 
                topics, 
                models, 
                fallbacks_list
            )
        )


def manual_chain_stream(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> Iterator[str]:
    print(f"Input: {topic}")
    prompt_value = prompt_template.format(topic=topic)

    if model == "chat_openai":
        print(f"Full prompt: {prompt_value}")
        stream = openai.OpenAI().chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_value}],
            stream=True,
        )
        for response in stream:
            content = response.choices[0].delta.content
            if content is not None:
                yield content
    elif model == "openai":
        print(f"Full prompt: {prompt_value}")
        stream = openai.OpenAI().completions.create(
            model="gpt-3.5-turbo-instruct", 
            prompt=prompt_value, 
            stream=True
        )
        for response in stream:
            yield response.choices[0].text
    elif model == "anthropic":
        prompt_value = f"Human:\n\n{prompt_value}\n\nAssistant:"
        print(f"Full prompt: {prompt_value}")
        stream = anthropic.Anthropic().completions.create(
            model="claude-2", 
            prompt=prompt_value, 
            max_tokens_to_sample=256, 
            stream=True,
        )
        for response in stream:
            yield response.completion
    else:
        raise ValueError(
            f"Invalid model {model}."
            "Should be one of chat_openai, openai, anthropic."
        )


async def manual_chain_async(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> str:
    # You get the idea :)
    ...


async def manual_chain_async_batch(
    topics: List[str], *, model: str = "chat_openai"
) -> List[str]:
    ...


async def manual_chain_async_stream(
    topic: str, *, model: str = "chat_openai"
) -> Iterator[str]:
    ...


def manual_chain_stream_with_fallbacks(
    topic: str, 
    *, 
    model: str = "chat_openai", 
    fallbacks: Tuple[str] = ("anthropic",),
) -> Iterator[str]:
    ...

</TabItem>

<TabItem value="lcel" label="LCEL">

In [ ]:
import os

from langchain.chat_models import ChatAnthropic, ChatOpenAI
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = chat_openai.with_fallbacks([anthropic]).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="chat_openai",
    openai=openai,
    anthropic=anthropic,
)

chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

</TabItem>

</Tabs>
